In [1]:
import pandas as pd
import numpy as np


In [3]:
df = pd.read_csv("../data/benin-malanville.csv", parse_dates=["Timestamp"])
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525600 entries, 0 to 525599
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Timestamp      525600 non-null  datetime64[ns]
 1   GHI            525600 non-null  float64       
 2   DNI            525600 non-null  float64       
 3   DHI            525600 non-null  float64       
 4   ModA           525600 non-null  float64       
 5   ModB           525600 non-null  float64       
 6   Tamb           525600 non-null  float64       
 7   RH             525600 non-null  float64       
 8   WS             525600 non-null  float64       
 9   WSgust         525600 non-null  float64       
 10  WSstdev        525600 non-null  float64       
 11  WD             525600 non-null  float64       
 12  WDstdev        525600 non-null  float64       
 13  BP             525600 non-null  int64         
 14  Cleaning       525600 non-null  int64         
 15  

In [7]:
print("The description of the numeric columns:")
print(df.describe())
print("---------------------------------------")
print("The description of the categorical columns:")
if not df.select_dtypes(include='object').empty:
    print(df.describe(include=["object"]))
else:
    print("No object (categorical/string) columns found.")
print("----------------------------------------")
print("The description of all columns")
print(df.describe(include='all'))
print("----------------------------------------")
print("An overview of the dataset: the first 5 rows")
print(df.head())
print("-----------------------------------------")
print("An overview of the dataset: the last 5 rows")
print(df.tail())
print("-----------------------------------------")
print("An overview of the dataset: a random sample of 5 rows")
print(df.sample(5))
print("-----------------------------------------")
print("An overview of the dataset: the shape of the dataset")
print(df.info())
print("------------------------------------------")
print("Check for missing values:")
print(df.isna().sum())

The description of the numeric columns:
                           Timestamp            GHI            DNI  \
count                         525600  525600.000000  525600.000000   
mean   2022-02-07 12:00:30.000000512     240.559452     167.187516   
min              2021-08-09 00:01:00     -12.900000      -7.800000   
25%              2021-11-08 06:00:45      -2.000000      -0.500000   
50%              2022-02-07 12:00:30       1.800000      -0.100000   
75%              2022-05-09 18:00:15     483.400000     314.200000   
max              2022-08-09 00:00:00    1413.000000     952.300000   
std                              NaN     331.131327     261.710501   

                 DHI           ModA           ModB           Tamb  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      115.358961     236.589496     228.883576      28.179683   
min       -12.600000       0.000000       0.000000      11.000000   
25%        -2.100000       0.000000       0.000000   